Example of multi-tenant user access and management on Waii

Prequisite first: create two postgres databases, which will be used by the example later

Run `sudo -u postgres psql` to open psql

```
-- Connect to PostgreSQL as the postgres superuser

-- Create the first user
CREATE USER waii_test_user1 WITH PASSWORD 'password1';

-- Create the first database owned by waii_test_user1
CREATE DATABASE waii_test_db1 OWNER waii_test_user1;

-- Create the second user
CREATE USER waii_test_user2 WITH PASSWORD 'password2';

-- Create the second database owned by user2
CREATE DATABASE waii_test_db2 OWNER waii_test_user2;

-- Grant privileges (optional but recommended)
GRANT ALL PRIVILEGES ON DATABASE waii_test_db1 TO waii_test_user1;
GRANT ALL PRIVILEGES ON DATABASE waii_test_db2 TO waii_test_user2;
```

You can use other databases (such as snowflake, all the examples below is exchangeabe)

In [3]:
!pip uninstall -y waii-sdk-py
!pip install --upgrade waii-sdk-py pydantic==2.10.3

Found existing installation: waii-sdk-py 1.29.2
Uninstalling waii-sdk-py-1.29.2:
  Successfully uninstalled waii-sdk-py-1.29.2
  Using cached waii_sdk_py-1.29.2-py3-none-any.whl.metadata (2.5 kB)
Using cached waii_sdk_py-1.29.2-py3-none-any.whl (50 kB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Now we are going to add some users
# First, let's import all libraries
# Import the main Waii SDK client
from waii_sdk_py import Waii

# Import specific modules
from waii_sdk_py.chat import *
from waii_sdk_py.query import *
from waii_sdk_py.database import *
from waii_sdk_py.semantic_context import *
from waii_sdk_py.chart import *
from waii_sdk_py.history import *
from waii_sdk_py.user import *

# Create admin client
WAII_API_KEY = 'test'
WAII_URL = 'http://localhost:3456/api/'

admin_client = Waii()
admin_client.initialize(api_key=WAII_API_KEY, url=WAII_URL)

Important thing to check before you start: you must enable api-key-auth when you start Waii service.

1) If you are using Waii SaaS deployment: you can skip this check.
2) If you are using Docker or K8s deployment, you need to enable it (`api_key_auth_enabled`)
   Check if it is enabled: 

   Open Waii UI using icognito window, you should see it pop up and ask you for an API key to login.

   [ ] check

In [5]:
# create an organization, let's call it fruit org
admin_client.user.create_org(
    CreateOrganizationRequest(
        organization=Organization(
            id='fruit-org',
            name='Fruit Org'
        )
    )
)

# then create a banana tenant
admin_client.user.create_tenant(
    CreateTenantRequest(
        tenant=Tenant(
            id='banana-tenant',
            name='Banana Tenant',
            org_id='fruit-org'
        )
    )
)


# then create a apple tenant
admin_client.user.create_tenant(
    CreateTenantRequest(
        tenant=Tenant(
            id='apple-tenant',
            name='Apple Tenant',
            org_id='fruit-org'
        )
    )
)

# then create a user for banana tenant
admin_client.user.create_user(
    CreateUserRequest(
        user=User(
            id='banana-user',
            name='Banana User',
            tenant_id='banana-tenant',
            org_id='fruit-org'
        )
    )
)

# then create a user for apple tenant
admin_client.user.create_user(
    CreateUserRequest(
        user=User(
            id='apple-user',
            name='Apple User',
            tenant_id='apple-tenant',
            org_id='fruit-org'
        )
    )
)

# now we have 3 users:
# - admin user
# - banana user
# - apple user

CommonResponse()

In [6]:
# try to list these users
current_users = admin_client.user.list_users(ListUsersRequest(lookup_org_id='fruit-org')).users
for _u in current_users:
    print(_u)
    
# you should see two users:
# - banana user
# - apple user

id='banana-user' name='Banana User' tenant_id='banana-tenant' org_id='fruit-org' variables=None roles=[]
id='apple-user' name='Apple User' tenant_id='apple-tenant' org_id='fruit-org' variables=None roles=[]


In [7]:
# update the users
# add waii-api-user role to banana user and apple user (keep other fields same)
admin_client.user.update_user(
    UpdateUserRequest(
        user=User(
            id='banana-user',
            tenant_id='banana-tenant',
            org_id='fruit-org',
            name='Banana User',
            roles=['waii-org-admin-user']
        )
    )
)
admin_client.user.update_user(
    UpdateUserRequest(
        user=User(
            id='apple-user',
            name='Apple User',
            roles=['waii-org-admin-user'],
            tenant_id='apple-tenant',
            org_id='fruit-org'
        )
    )
)
        
# get users again, you should be able to see the roles updated
# you should be able to see roles=['waii-api-user'] from the response
current_users = admin_client.user.list_users(ListUsersRequest(lookup_org_id='fruit-org')).users
for _u in current_users:
    print(_u)

id='banana-user' name='Banana User' tenant_id='banana-tenant' org_id='fruit-org' variables=None roles=['waii-org-admin-user']
id='apple-user' name='Apple User' tenant_id='apple-tenant' org_id='fruit-org' variables=None roles=['waii-org-admin-user']


In [8]:
# now we try to create api keys for banana user and apple user
# create api key for banana user
# here it uses 'impersonate_user' to create api key for banana user
# note that only super admin / org admin can do impersonate
# the impersonate context will be cleared after the with block
with admin_client.impersonate_user('banana-user'):
    try:
        banana_api_key = admin_client.user.create_access_key(
            CreateAccessKeyRequest(
                # the name is allow you to create named-api-key, you can create multiple api keys for the same user
                # currently all the api keys under the same user are treated as the same user, but in the future we will
                # support different api key permission for same user
                name='default'
            )
        )[0].access_key
    except:
        # do a try catch here because the api key might already exists, so we just get it.
        banana_api_key = admin_client.user.list_access_keys(GetAccessKeyRequest()).access_keys[0].access_key

# create api key for apple user
with admin_client.impersonate_user('apple-user'):
    try:
        apple_api_key = admin_client.user.create_access_key(
            CreateAccessKeyRequest(
                name='default'
            )
        )[0].access_key
    except:
        apple_api_key = admin_client.user.list_access_keys(GetAccessKeyRequest()).access_keys[0].access_key

In [9]:
# now we have 2 api keys:
# - banana-api-key
# - apple-api-key

# we can create multiple waii clients with different api keys
# and use them to access different tenants

# create a waii client for banana user
banana_client = Waii()
banana_client.initialize(api_key=banana_api_key, url=WAII_URL)

# create a waii client for apple user
apple_client = Waii()
apple_client.initialize(api_key=apple_api_key, url=WAII_URL)

# now try to get user info from banana client
banana_user_info = banana_client.user.get_user_info(GetUserInfoRequest())
print(banana_user_info)

# try to get user info from apple client
apple_user_info = apple_client.user.get_user_info(GetUserInfoRequest())
print(apple_user_info)

# you should see the user info from banana client returns the banana user info, 
# and the apple client returns the apple user info

# Note here: the waii client is very lightweight, it does not hold any state (just a few string variables to store the API key and url)
# so you can create as many as you want, and use them interchangeably.
# It also doesn't have any long-running session, so you don't need to care about the session expiration // resource consumption.

id='banana-user' name='Banana User' email='banana-user' roles=['waii-org-admin-user'] permissions=['write:access_key', 'read:org', 'write:similarity-search-index', 'usage:api', 'read:liked-queries', 'write:semantic-context', 'read:tenant', 'publish:semantic-context', 'publish:access-rules', 'push:databases', 'write:user', 'publish:liked-queries', 'write:databases', 'read:similarity-search-index', 'usage:impersonation', 'write:tenant', 'write:liked-queries', 'read:usage-report', 'read:user', 'read:databases', 'read:semantic-context']
id='apple-user' name='Apple User' email='apple-user' roles=['waii-org-admin-user'] permissions=['write:access_key', 'read:org', 'write:similarity-search-index', 'usage:api', 'read:liked-queries', 'write:semantic-context', 'read:tenant', 'publish:semantic-context', 'publish:access-rules', 'push:databases', 'write:user', 'publish:liked-queries', 'write:databases', 'read:similarity-search-index', 'usage:impersonation', 'write:tenant', 'write:liked-queries', 'r

In [10]:
banana_client.database.get_connections()

GetDBConnectionResponse(connectors=[DBConnection(key='waii://waii@host/waii-usage-reporting', db_type='postgresql', description=None, account_name=None, username=None, password=None, database='waii-usage-reporting', warehouse=None, role=None, path=None, host=None, port=None, parameters=None, sample_col_values=True, push=False, db_content_filters=None, embedding_model=None, always_include_tables=None, alias=None, db_access_policy=DBAccessPolicy(read_only=False, allow_access_beyond_db_content_filter=True, allow_access_beyond_search_context=True), host_alias=None, user_alias=None, db_alias=None, client_email=None, content_filters=None, sample_filters=None, external_authentication_uri=None, waii_user_id=None, secure=True, need_external_authentication=None), DBConnection(key='snowflake://WAII@gqobxjv-bhb91428/MOVIE_DB?role=WAII_USER_ROLE&warehouse=COMPUTE_WH', db_type='snowflake', description=None, account_name='gqobxjv-bhb91428', username='WAII', password=None, database='MOVIE_DB', warehou

In [11]:
apple_client.database.get_connections()

GetDBConnectionResponse(connectors=[DBConnection(key='waii://waii@host/waii-usage-reporting', db_type='postgresql', description=None, account_name=None, username=None, password=None, database='waii-usage-reporting', warehouse=None, role=None, path=None, host=None, port=None, parameters=None, sample_col_values=True, push=False, db_content_filters=None, embedding_model=None, always_include_tables=None, alias=None, db_access_policy=DBAccessPolicy(read_only=False, allow_access_beyond_db_content_filter=True, allow_access_beyond_search_context=True), host_alias=None, user_alias=None, db_alias=None, client_email=None, content_filters=None, sample_filters=None, external_authentication_uri=None, waii_user_id=None, secure=True, need_external_authentication=None), DBConnection(key='snowflake://WAII@gqobxjv-bhb91428/MOVIE_DB?role=WAII_USER_ROLE&warehouse=COMPUTE_WH', db_type='snowflake', description=None, account_name='gqobxjv-bhb91428', username='WAII', password=None, database='MOVIE_DB', warehou